In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import pandas as pd
from urllib.parse import urljoin
def crawl_festival_data(driver, detail_url):
    # 상세 페이지로 이동
    driver.get(detail_url)
    # 페이지 로딩을 기다림 (3초 기다리고 조절 가능)
    driver.implicitly_wait(3)
    # 현재 페이지의 소스 코드
    current_page_source = driver.page_source
    # BeautifulSoup을 사용하여 HTML 파싱
    soup = BeautifulSoup(current_page_source, 'html.parser')
    # 데이터를 저장할 딕셔너리 초기화
    festival_data = {}
    # 제목 추출
    title_tag = soup.find('div', class_='view_title')
    if title_tag:
        festival_data['Title'] = title_tag.get_text(strip=True)
    else:
        festival_data['Title'] = 'N/A'
    # 이미지 추출
    img_tag = soup.find('div', class_='culture_view_img').find('img')
    if img_tag:
        festival_data['Image_URL'] = urljoin('https://www.mcst.go.kr', img_tag['src'])
    # 기타 정보 추출
    dl_elements = soup.find_all('dl')
    for dl in dl_elements:
        dt = dl.find('dt').get_text(strip=True)
        dd = dl.find('dd').get_text(strip=True)
        # 일부 정보는 특정한 전처리가 필요할 수 있음
        if dt == '개최기간':
            # 개최기간 예시로 정규화
            dd = dd.replace('.', '-')
        # 추가 정보 추출 (예시: 개최지역, 축제성격, 관련 누리집 등)
        if dt in ['개최지역', '축제성격', '관련 누리집', '축제장소', '요금', '주최/주관기관', '문의']:
            festival_data[dt] = dd
    # 상세 내용 추출
    view_con_tag = soup.find('div', class_='view_con')
    if view_con_tag:
        festival_data['Description'] = view_con_tag.get_text(strip=True)
    return festival_data
# Chrome WebDriver 설정
driver = webdriver.Chrome()
# 예시 URL
url = "https://www.mcst.go.kr/kor/s_culture/festival/festivalList.jsp?pMenuCD=&pCurrentPage=1&pSearchType=01&pSearchWord=&pSeq=12067&pSido=&pOrder=01up&pPeriod=&fromDt=&toDt="
# 상세 페이지 링크 가져오기
driver.get(url)
detail_links = [link.get_attribute('href') for link in driver.find_elements(By.CSS_SELECTOR, 'a.go')]
# 각 상세 페이지에서 정보 추출
festival_data_list = []
for detail_link in detail_links:
    festival_data = crawl_festival_data(driver, detail_link)
    festival_data_list.append(festival_data)
# WebDriver 종료
driver.quit()
# 결과를 데이터프레임으로 변환
result_df = pd.DataFrame(festival_data_list)
# 결과 출력
print(result_df)

Empty DataFrame
Columns: []
Index: []
